Importando Bibliotecas

In [1]:
!pip3 install torch torchaudio torchvision torchtext torchdata
!pip3 install pyDecision

Defaulting to user installation because normal site-packages is not writeable



[notice] A new release of pip is available: 24.3.1 -> 25.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip


Defaulting to user installation because normal site-packages is not writeable



[notice] A new release of pip is available: 24.3.1 -> 25.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [2]:
import torch
import pickle
import random
import time
import zipfile
import os
import json
import torch
import pickle
from torch.utils.data import DataLoader
from torch.utils.data.sampler import SubsetRandomSampler
import numpy as np
import torch.nn as nn
import torchvision
import torch.optim as optim
import torch.nn.functional as F
import torchvision.models as models
from torchvision.models.inception import InceptionOutputs
from torch.utils.data import DataLoader
from torchvision import datasets, transforms
from pyDecision.algorithm import ahp_method
from sklearn.metrics import accuracy_score, precision_score, recall_score

Carregando DATASET

In [3]:
zip_dataset_warm = '/content/dataset-resized'

In [4]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Usando dispositivo:", device)

Usando dispositivo: cuda


In [ ]:
def cifar_10(n_valid=0.2, batch_size=64, num_workers=4):
    """10 Classes"""
    
    transform_train = transforms.Compose([transforms.ToTensor(),
                                    transforms.Resize((224, 224)),
                                    transforms.Normalize([0.4914, 0.4822, 0.4465],[0.2023, 0.1994, 0.2010])])
    
    transform_test = transforms.Compose([
        transforms.ToTensor(),
        transforms.Normalize([0.4914, 0.4822, 0.4465], [0.2023, 0.1994, 0.2010])  
    ])
    
    train_data = torchvision.datasets.CIFAR10(root='./data', train=True,
                                        download=True, transform=transform_train)
    test_data = torchvision.datasets.CIFAR10(root='./data', train=False,
                                        download=True, transform=transform_test)

    n_train = len(train_data)
    indices = list(range(n_train))
    np.random.shuffle(indices)
    split = int(np.floor(n_valid * n_train))
    train_idx, valid_idx = indices[split:], indices[:split]

    # Define samplers for obtaining training and validation
    train_sampler = SubsetRandomSampler(train_idx)
    valid_sampler = SubsetRandomSampler(valid_idx)

    # Prepare data loaders (combine dataset and sampler)
    trainLoader = torch.utils.data.DataLoader(train_data,
                                            batch_size = batch_size,
                                            sampler = train_sampler,
                                            num_workers = num_workers)

    validLoader = torch.utils.data.DataLoader(train_data,
                                            batch_size = batch_size,
                                            sampler = valid_sampler,
                                            num_workers = num_workers)

    testLoader = torch.utils.data.DataLoader(test_data,
                                            batch_size = batch_size,
                                            num_workers = num_workers)

    classes = train_data.classes
    return trainLoader, validLoader, testLoader, classes

##### AJUSTA PARA CIFAR-10 SUBSAMPLE #####################

def cifar_10_subsample(batch_size=64, num_workers=4, class_labels=['cat', 'dog']):
    """CIFAR-10 with only two selected classes for warm"""
    transform = transforms.Compose([transforms.ToTensor(),
                                    transforms.Resize((224, 224)),
                                    transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010))])
    
    # Load the entire CIFAR-10 test dataset
    test_data = torchvision.datasets.CIFAR10(root='./data', train=False,
                                             download=True, transform=transform)

    # Filter out only the specified classes
    test_indices = [i for i, label in enumerate(test_data.targets) if test_data.classes[label] in class_labels]

    # Map the selected classes to new labels (0 and 1)
    test_data.targets = [test_data.targets[i] for i in test_indices]
    test_data.data = test_data.data[test_indices]

    label_map = {test_data.classes.index(class_labels[0]): 0,
                 test_data.classes.index(class_labels[1]): 1}

    test_data.targets = [label_map[label] for label in test_data.targets]

    # Prepare data loader for test data
    testLoader = DataLoader(test_data, 
                            batch_size=batch_size,
                            num_workers=num_workers)
    
    classes = class_labels
    
    return testLoader, classes

#### Definindo os modelos

In [ ]:
def get_efficientnet_b0(num_classes=10):
    model = models.efficientnet_b0(weights=models.EfficientNet_B0_Weights.IMAGENET1K_V1)
    if hasattr(model.classifier, 'in_features'):
        in_features = model.classifier.in_features
    elif hasattr(model.classifier, 'fc'):
        in_features = model.classifier.fc.in_features
    else:
        in_features = model.classifier[-1].in_features
    model.classifier[-1] = nn.Linear(in_features, num_classes)
    return model.to(device)

def get_mobilenet_v2(num_classes=10):
    model = models.mobilenet_v2(weights=models.MobileNet_V2_Weights.IMAGENET1K_V1)
    model.classifier[1] = nn.Linear(model.classifier[1].in_features, num_classes)
    return model.to(device)

def get_resnet50(num_classes=10):
    model = models.resnet50(weights=models.ResNet50_Weights.DEFAULT)
    model.fc = nn.Linear(model.fc.in_features, num_classes)
    return model.to(device)

def get_inception_v3(num_classes=10):
    model = models.inception_v3(weights=models.Inception_V3_Weights.IMAGENET1K_V1, aux_logits=True)
    model.AuxLogits.fc = nn.Linear(model.AuxLogits.fc.in_features, num_classes)
    model.fc = nn.Linear(model.fc.in_features, num_classes)
    return model.to(device)

def get_vgg16(num_classes=10):
    #model = models.vgg19(weights=models.VGG19_Weights.IMAGENET1K_V1)
    #model = models.vgg13(weights=models.VGG13_Weights.IMAGENET1K_V1)
    #model = models.vgg16(weights=models.VGG16_Weights.IMAGENET1K_V1)
    model = models.vgg16_bn(weights=models.VGG16_BN_Weights.IMAGENET1K_V1)
    # Congelar as camadas convolucionais
    #for param in model.features.parameters():
    #    param.requires_grad = False
    model.classifier[6] = nn.Linear(model.classifier[6].in_features, num_classes)
    return model.to(device)


#### Definição das funções do Método OACE

In [7]:
metrics_a = ["precision", "accuracy", "recall"]
metrics_c = ["mtp", "tpi", "ms"]
epsilon = 1e-5 # Parâmetro da padronização para evitar divisão por zero
### AHP Method ###
weight_derivation = 'geometric' # 'mean'; 'geometric' or 'max_eigen'
dataset = np.array([# Dataset for assertiveness metrics (P -> A -> R)
  #P      A      R
  [1  ,   5,     7   ],   #P
  [1/5,   1,     3   ],   #A
  [1/7,   1/3,   1   ],   #R
])
dataset = np.array([ # Dataset for cost metrics (MTP -> TPI -> MS)
  #MTP    TPI    MS
  [  1,     5,   7   ],   #MTP
  [1/5,     1,   3   ],   #TPI
  [1/7,   1/3,   1   ],   #MS
])

weights, rc = ahp_method(dataset, wd = weight_derivation)
w1, w2, w3 = weights[0], weights[1], weights[2]
wa = [w1, w2, w3] # Precision, Acuraccy, Recall
wc = [w1, w2, w3] # MTP, TPI, MS

def get_max_min_metrics(metrics_dict):
    """Calcula os máximos e mínimos das métricas de custo resultante do aquecimento dos modelos"""
    metricas_custo = ['mtp', 'tpi', 'ms']

    max_custo = {metrica: float('-inf') for metrica in metricas_custo}
    min_custo = {metrica: float('inf') for metrica in metricas_custo}

    for model_name, metrics in metrics_dict.items():
        for metrica in metricas_custo:
            valor = metrics[metrica]
            if valor > max_custo[metrica]:
                max_custo[metrica] = valor
            if valor < min_custo[metrica]:
                min_custo[metrica] = valor

    # Convertendo os dicionários de máximos e mínimos em listas simples
    max_custo_list = [max_custo[metrica] for metrica in metricas_custo]
    min_custo_list = [min_custo[metrica] for metrica in metricas_custo]

    return max_custo_list, min_custo_list

In [8]:
def N(value, max_value, min_value):
    """Função de normalização min-max, com clamp para garantir valores entre 0 e 1 e logs para valores fora do intervalo."""
    if max_value is None or min_value is None or max_value == min_value:
        return 0.0
    normalized = (value - min_value) / (max_value - min_value)
    return max(0.0, min(1.0, normalized))  # Clampa entre 0 e 1

def N_cost(value, max_value, min_value):
    if max_value is None or min_value is None or max_value == min_value:
        return 0.0
    normalized = (max_value - value) / (max_value - min_value)
    return max(0.0, min(1.0, normalized))  

def A(metrics, wa, metricas_a, maximos_a, minimos_a):
    """Calcula a assertividade normalizada."""
    a_i = [metrics['assertividade'][metrica] for metrica in metricas_a]
    normalized_values = [N(a, maximo, minimo) for a, maximo, minimo in zip(a_i, maximos_a, minimos_a)]
    return sum([n * w for n, w in zip(normalized_values, wa)])

def C(metrics, wc, metricas_c, maximos_c, minimos_c):
    """Calcula o custo normalizado."""
    c_i = [metrics['custo'][metrica] for metrica in metricas_c]
    normalized_values = [N_cost(c, maximo, minimo) for c, maximo, minimo in zip(c_i, maximos_c, minimos_c)]
    return sum([n * w for n, w in zip(normalized_values, wc)])

def F_score(lambda_, assertiveness, cost, max_score, min_score):
    """Calcula o score do método e normaliza."""
    score = lambda_ * assertiveness + (1 - lambda_) * cost
    return N(score, max_score, min_score)

def calculo_maximos_minimos(accumulated_metrics, metricas, tipo):
    """Calcula os máximos e mínimos para normalização das métricas."""
    if not accumulated_metrics:
        return [0.0] * len(metricas), [0.0] * len(metricas)  # Valores padrão para evitar erros iniciais

    valores = {metrica: [] for metrica in metricas}
    for metrics in accumulated_metrics:
        for metrica in metricas:
            valores[metrica].append(float(metrics[tipo][metrica]))

    maximos = [np.max(valores[metrica]) for metrica in metricas]
    minimos = [np.min(valores[metrica]) for metrica in metricas]
    return maximos, minimos

def calcular_metricas_oace(iteration_metrics, accumulated_metrics, lambda_, wa, wc, metricas_a, metricas_c, maximos_a=None, minimos_a=None, maximos_c=None, minimos_c=None):
    print("\n🔎 INICIANDO A AVALIAÇÃO DO OACE PARA ITERAÇÃO ATUAL")
    print("iteration_metrics: ", iteration_metrics)
    print("accumulated_metrics: ", accumulated_metrics)

    if not iteration_metrics:
        return None
    if maximos_a is None or minimos_a is None:
        maximos_a, minimos_a = calculo_maximos_minimos(accumulated_metrics, metricas_a, "assertividade")
        
    print("maximos_a: ", maximos_a)
    print("minimos_a: ", minimos_a)
    print("maximos_c (fixos do aquecimento): ", maximos_c)
    print("minimos_c (fixos do aquecimento): ", minimos_c)

    max_score = lambda_ * 1.0 + (1 - lambda_) * 1.0  # Máximo teórico
    min_score = lambda_ * 0.0 + (1 - lambda_) * 0.0  # Mínimo teórico

    assertividade = A(iteration_metrics, wa, metricas_a, maximos_a, minimos_a)
    print("🔹 A (Assertividade Normalizada): ", assertividade)
    custo = C(iteration_metrics, wc, metricas_c, maximos_c, minimos_c)
    print("🔹 C (Custo Normalizado): ", custo)
    score = F_score(lambda_, assertividade, custo, max_score, min_score)
    print("🔹 S (Score OACE): ", score)

    return {
        "model_name": iteration_metrics["model_name"],
        "A": assertividade,
        "C": custo,
        "Score": score,
        "solution": iteration_metrics["solution"]
    }

#### Código para hyperparametrização do método com random walk

Gerando as soluções com o random walk (alterar para configurar a semente random.seed())

In [9]:
def generate_solution(model_list):
    """Gera uma solução inicial para o Random Walk, em termos de taxa de aprendizado (lr) e modelo"""
    lr = random.uniform(1e-4, 1e-2)
    model_index = random.randint(0, len(model_list) - 1)  # Escolhe um modelo da lista
    return [lr, model_index]

def random_walk_step(solution, model_list, step_size=0.1):
    """Executa um passo do Random Walk para explorar novas soluções"""
    new_solution = solution.copy()
    new_solution[0] = min(max(new_solution[0] + random.uniform(-step_size * new_solution[0], step_size * new_solution[0]), 1e-4), 1e-2)
    new_solution[1] = random.randint(0, len(model_list) - 1)  # Seleciona outro modelo aleatoriamente
    return new_solution

Funções para treinamento e validação dos modelos

In [37]:
def train_models(model, trainLoader, validLoader, criterion, optimizer, epochs=30, early_stopping_rounds=5):
    """Treina o modelo e monitora a perda de validação para aplicar early stopping."""
    best_val_loss = float('inf')
    no_improvement_count = 0

    for epoch in range(1, epochs + 1):

        train_loss = 0.0
        valid_loss = 0.0

        model.train()
        for data, target in trainLoader:
            data, target = data.to(device), target.to(device)

            # Redimensionamento dinâmico se necessário
            if isinstance(model, models.Inception3):
                data = F.interpolate(data, size=(299, 299), mode='bilinear', align_corners=False)

            optimizer.zero_grad()
            output = model(data)

            # Verifica se a saída é do tipo InceptionOutputs
            if isinstance(output, InceptionOutputs):
                output = output.logits  # Usa apenas a saída principal

            loss = criterion(output, target)
            loss.backward()
            optimizer.step()
            train_loss += loss.item() * data.size(0)

        model.eval()
        with torch.no_grad():
            for data, target in validLoader:
                data, target = data.to(device), target.to(device)

                 # Redimensionamento dinâmico se necessário
                if isinstance(model, models.Inception3):
                    data = F.interpolate(data, size=(299, 299), mode='bilinear', align_corners=False)

                output = model(data)
                # Verifica se a saída é do tipo InceptionOutputs
                if isinstance(output, InceptionOutputs):
                    output = output.logits  # Usa apenas a saída principal
                loss = criterion(output, target)
                valid_loss += loss.item() * data.size(0)

        # calculate average losses
        train_loss = train_loss/len(trainLoader.dataset)
        valid_loss = valid_loss/len(validLoader.dataset)

        print('Epoch: {} \tTraining Loss: {:.6f} \tValidation Loss: {:.6f}'.format(
            epoch, train_loss, valid_loss))

        if valid_loss < best_val_loss:
            best_val_loss = valid_loss
            no_improvement_count = 0
        else:
            no_improvement_count += 1

        if no_improvement_count >= early_stopping_rounds:
            print(f"Early stopping after {epoch} epochs due to no improvement.")
            break

    return model

def evaluate_solution(model, trainLoader, testLoader, validLoader, criterion, optimizer, dataset_name):
    """
    - Avalia uma solução de modelo treinado medindo sua precisão, acurácia, recall, tempo de inferência, tamanho e número de parâmetros.
    - Esta função treina o modelo, realiza inferências no conjunto de testes e calcula várias métricas de desempenho, incluindo assertividade e custo computacional.
    """
    model = train_models(model, trainLoader, validLoader, criterion, optimizer)
    model.eval()
    all_preds = []
    all_labels = []
    inference_times = []

    with torch.no_grad():
        for inputs, target in testLoader:
            inputs, target = inputs.to(device), target.to(device)

            # Redimensionamento dinâmico se necessário
            if isinstance(model, models.Inception3):
                inputs = F.interpolate(inputs, size=(299, 299), mode='bilinear', align_corners=False)

            start_time = time.time()
            outputs = model(inputs)
            inference_time = time.time() - start_time
            _, preds = torch.max(outputs, 1)
            all_preds.extend(preds.cpu().numpy())
            all_labels.extend(target.cpu().numpy())
            inference_times.append(inference_time)

    # Defina o valor de 'average' com base no tipo de dataset
    if dataset_name == "Chest X-Ray":
        average_type = 'binary'
    else:
        average_type = 'macro'

    precision = precision_score(all_labels, all_preds, average=average_type, zero_division=0)
    accuracy = accuracy_score(all_labels, all_preds)
    recall = recall_score(all_labels, all_preds, average=average_type, zero_division=0)# No dataset char x-ray average binary deve ser utilizada, para o restanto, micro ou average

    avg_inference_time = sum(inference_times) / len(inference_times)
    num_params = sum(p.numel() for p in model.parameters()) 
    model_size = sum(p.element_size() * p.numel() for p in model.parameters()) / (1024 ** 2)

    return float(precision), accuracy, float(recall), avg_inference_time, model_size, num_params


Função para Aquecimento dos modelos

In [12]:
def warm_calculate_metrics(model, dataloader, device):
    """Calcula as métricas de custo no aquecimento para um modelo dado"""
    model.eval()
    total_inference_time = 0.0

    inference_times = []

    with torch.no_grad():
        for inputs, target in dataloader:
            inputs, target = inputs.to(device), target.to(device)
            start_time = time.time()
            outputs = model(inputs)
            inference_time = time.time() - start_time
            _, preds = torch.max(outputs, 1)
            inference_times.append(inference_time)

    avg_inference_time = sum(inference_times) / len(inference_times)
    num_params = sum(p.numel() for p in model.parameters())
    model_size = sum(p.element_size() * p.numel() for p in model.parameters()) / (1024**2)  # Size in MB

    return num_params, avg_inference_time, model_size

def warm_up_models(models, dataloader, device):
    metrics = {}
    for model_name, get_model_func in models:
        model = get_model_func().to(device)
        num_params, avg_inference_time, model_size = warm_calculate_metrics(model, dataloader, device)
        metrics[model_name] = {
            'mtp': num_params,
            'tpi': avg_inference_time,
            'ms': model_size
        }
    return metrics

Checkpoint

In [13]:
def save_checkpoint(current_round, iteration, best_model_overall, best_score_overall, best_solution_overall,
                    metrics_per_iteration, oace_metrics_per_iteration, max_assertiveness_max, max_assertiveness_min, checkpoint_path):
    """Salva o estado atual do treinamento para continuar depois."""
    checkpoint_data = {
        'current_round': current_round,
        'iteration': iteration,
        'best_model_overall': best_model_overall,
        'best_score_overall': best_score_overall,
        'best_solution_overall': best_solution_overall,
        'metrics_per_iteration': metrics_per_iteration,  # Armazena todas as iterações
        'oace_metrics_per_iteration': oace_metrics_per_iteration,  # Armazena todas as métricas OACE
        'max_assertiveness_max': max_assertiveness_max,
        'max_assertiveness_min': max_assertiveness_min
    }
    with open(checkpoint_path, 'wb') as f:
        pickle.dump(checkpoint_data, f)
    print(f"✅ Checkpoint salvo na Rodada {current_round + 1}, Iteração {iteration + 1}.")

def load_checkpoint(checkpoint_path):
    """Carrega o checkpoint, se existir."""
    if os.path.exists(checkpoint_path):
        with open(checkpoint_path, 'rb') as f:
            checkpoint = pickle.load(f)
        return checkpoint
    else:
        return None

Função principal para execução do método (alterar para ajustar para nova metodologia)

In [31]:
def optimize_hyperparameters(models_list, trainloader, testloader, validLoader, classes, lbd, wa, wc, dataset_name, checkpoint_path, num_rounds=3, iterations_per_round=10, save_checkpoint_every=5):
    
    initial_seeds = [100, 600, 1100]  # Seeds fixas, pode ser [50, 150, 200], [100, 600, 1100] ou [1000, 2500, 4000]

    checkpoint = load_checkpoint(checkpoint_path)
    if checkpoint:
        current_round = checkpoint['current_round']
        iteration = checkpoint['iteration']
        best_model_overall = checkpoint['best_model_overall']
        best_score_overall = checkpoint['best_score_overall']
        best_solution_overall = checkpoint['best_solution_overall']
        metrics_per_iteration = checkpoint['metrics_per_iteration']  
        oace_metrics_per_iteration = checkpoint['oace_metrics_per_iteration']  
        maximos_a = checkpoint['max_assertiveness_max']  
        minimos_a = checkpoint['max_assertiveness_min']  

        print(f"🔄 Continuação do treinamento a partir da Rodada {current_round + 1}, Iteração {iteration + 1}...")
    else:
        current_round = 0
        iteration = 0  
        best_model_overall = None
        best_score_overall = float('-inf')
        best_solution_overall = None
        metrics_per_iteration = {}  # Dicionário para todas as iterações
        oace_metrics_per_iteration = {}  # Dicionário para todas as métricas OACE
        maximos_a, minimos_a = None, None  # Inicia como None para recalcular ou carregar do checkpoint

    with open('warm_up_metrics.pkl', 'rb') as f:
        warm_up_metrics = pickle.load(f)

    maximos_c, minimos_c = get_max_min_metrics(warm_up_metrics)

    accumulated_metrics = []  # Para calcular máximos/mínimos de assertividade
    total_iterations = num_rounds * iterations_per_round  # Total de iterações globais

    for round_idx in range(current_round, num_rounds):
        print(f"\n🔄 Iniciando Rodada {round_idx + 1}/{num_rounds} 🔄")
        base_seed = initial_seeds[round_idx] 
        best_model = None
        best_score = float('-inf')
        best_solution = None if not checkpoint or round_idx > current_round else best_solution_overall

        # Reinicia a solução se for um novo round após o checkpoint
        if round_idx > current_round or not checkpoint:
            solution = generate_solution(models_list)
        else:
            solution = best_solution_overall

        # Itera sobre as iterações desta rodada, incrementando a seed
        for local_iteration in range(iterations_per_round):
            global_iteration = round_idx * iterations_per_round + local_iteration  # Iteração global
            if global_iteration <= iteration and checkpoint:  # Pula iterações já processadas
                continue

            current_seed = base_seed + local_iteration  # Incrementa a seed dentro do round
            random.seed(current_seed)

            print(f"▶️ Iteration {global_iteration + 1}/{total_iterations} (Round {round_idx + 1}): model {solution[1]} e lr {solution[0]}")
            print(f"🔀 Random Seed Atual: {current_seed}")

            model_name, Model = models_list[solution[1]]
            model = Model(num_classes=len(classes)).to(device)
            optimizer = optim.Adam(model.parameters(), lr=solution[0])
            criterion = nn.CrossEntropyLoss()

            precision, accuracy, recall, avg_inference_time, model_size, num_params = evaluate_solution(
                model, trainloader, testloader, validLoader, criterion, optimizer, dataset_name)

            current_metrics = {
                "model_name": model_name,
                "assertividade": {"precision": precision, "accuracy": accuracy, "recall": recall},
                "custo": {"mtp": num_params, "tpi": avg_inference_time, "ms": model_size},
                "solution": {"lr": solution[0], "model_index": solution[1]}
            }

            accumulated_metrics.append(current_metrics)
            metrics_per_iteration[global_iteration] = current_metrics

            # Calcular OACE usando os custos fixos do aquecimento e os máximos/mínimos de assertividade carregados ou atualizados
            oace_metrics = calcular_metricas_oace(
                current_metrics,
                accumulated_metrics,
                lbd, wa, wc,
                ["precision", "accuracy", "recall"],
                ["mtp", "tpi", "ms"],
                maximos_a, minimos_a,
                maximos_c, minimos_c
            )
            oace_metrics_per_iteration[global_iteration] = oace_metrics

            print("Resultado do OACE por Iteração: ", oace_metrics)
            score = oace_metrics["Score"]

            # Atualizar o melhor modelo globalmente (sobre todas as iterações)
            if score > best_score_overall:
                best_score_overall = score
                best_model_overall = model_name
                best_solution_overall = solution.copy()

            solution = random_walk_step(solution, models_list)

            # Salvar checkpoint a cada 'save_checkpoint_every' iterações globais
            if (global_iteration + 1) % save_checkpoint_every == 0 or global_iteration == total_iterations - 1:
                save_checkpoint(
                    round_idx, global_iteration, best_model_overall, best_score_overall, best_solution_overall,
                    metrics_per_iteration, oace_metrics_per_iteration, maximos_a, minimos_a, checkpoint_path
                )

        # Atualizar máximos e mínimos de assertividade ao final de cada round
        current_max_a, current_min_a = calculo_maximos_minimos(accumulated_metrics, ["precision", "accuracy", "recall"], "assertividade")
        if maximos_a is None or minimos_a is None:
            maximos_a, minimos_a = current_max_a, current_min_a
        else:
            for i in range(len(current_max_a)):
                maximos_a[i] = max(maximos_a[i], current_max_a[i])
                minimos_a[i] = min(minimos_a[i], current_min_a[i])

    return best_model_overall, best_score_overall, best_solution_overall, metrics_per_iteration, oace_metrics_per_iteration

Código principal

In [32]:
datasets_options = {
        '1': 'Chest X-Ray',
        '2': 'CIFAR-10',
        '3': 'TrashNet'
    }

In [ ]:
#train_dir = '/content/drive/MyDrive/Método OACE/datasets/chest_xray/train'
#val_dir = '/content/drive/MyDrive/Método OACE/datasets/chest_xray/val'
#test_dir = '/content/drive/MyDrive/Método OACE/datasets/chest_xray/test'
#lbd = 0.75
#trainLoader, validLoader, testLoader, classes = chest_x_ray(train_dir, val_dir, test_dir)

#subsample_loader, subsample_classes = chest_x_ray_subsample(test_dir)
################################################################################
# CIFAR-10
lbd = 0.5
trainLoader, validLoader, testLoader, classes = cifar_10(batch_size=64)
dataset_name = datasets_options['2']
#subsample_loader, subsample_classes = cifar_10_subsample()
################################################################################
# TrashNet
#lbd = 0.25
#dataset_dir = 'datasets/dataset-resized'
#trainLoader, validLoader, testLoader, classes = trashNet(dataset_dir)
#ataset_name = datasets_options['3']
#subsample_loader, subsample_classes = trashNet_subsample(dataset_dir)
################################################################################

Files already downloaded and verified
Files already downloaded and verified


In [33]:
len(trainLoader)

625

In [34]:
for images, labels in trainLoader:
    print(f"Batch de imagens shape: {images.shape}")
    print(f"Batch de rótulos shape: {labels.shape}")
    break

Batch de imagens shape: torch.Size([64, 3, 32, 32])
Batch de rótulos shape: torch.Size([64])


In [ ]:
models_list = [
    ("EfficientNetB0", get_efficientnet_b0),
    ("MobileNetV2", get_mobilenet_v2),
    ("ResNet50", get_resnet50),
    ("InceptionV3", get_inception_v3),
    ("VGG16", get_vgg16)
]

wa = [0.731, 0.188, 0.081]  # Pesos para Precision, Accuracy, Recall
wc = [0.731, 0.188, 0.081]  # Pesos para MTP, TPI, MS

print("INFORMAÇÕES SOBRE O AMBIENTE DE EXECUÇÃO:")
print(f"PyTorch version: {torch.__version__}")
print(f"CUDA available: {torch.cuda.is_available()}")
if torch.cuda.is_available():
    print(f"CUDA version: {torch.version.cuda}")
    print(f"Number of GPUs: {torch.cuda.device_count()}")
    print(f"Current GPU: {torch.cuda.current_device()}")
    print(f"GPU name: {torch.cuda.get_device_name(torch.cuda.current_device())}")

print("\nIniciando o aquecimento dos modelos...\n")
subsample_loader, subsample_classes = cifar_10_subsample()  # Atualize para o caminho correto
print("Classes dos dados para aquecimento:", subsample_classes)
for images, labels in subsample_loader:
    print(f"Batch de imagens shape: {images.shape}")
    print(f"Batch de rótulos shape: {labels.shape}")
    break
warm_up_metrics = warm_up_models(models_list, subsample_loader, device)
with open('warm_up_metrics.pkl', 'wb') as f:
    pickle.dump(warm_up_metrics, f)
    
print("Warm-up concluído e métricas salvas.")

INFORMAÇÕES SOBRE O AMBIENTE DE EXECUÇÃO:
PyTorch version: 2.3.1+cu121
CUDA available: True
CUDA version: 12.1
Number of GPUs: 1
Current GPU: 0
GPU name: NVIDIA GeForce RTX 3050 6GB Laptop GPU

Iniciando o aquecimento dos modelos...

Classes dos dados para aquecimento: ['glass', 'plastic']
Batch de imagens shape: torch.Size([16, 3, 224, 224])
Batch de rótulos shape: torch.Size([16])
Warm-up concluído e métricas salvas.


In [38]:
checkpoint_path = "checkpoint.pkl"
best_model, best_score, best_solution, metrics, oace_metrics = optimize_hyperparameters(models_list, 
                                                                                        trainLoader, testLoader, validLoader, 
                                                                                        classes, lbd, wa, wc,
                                                                                        dataset_name, checkpoint_path, 
                                                                                        num_rounds=3, iterations_per_round=5, 
                                                                                        save_checkpoint_every=3)


🔄 Iniciando Rodada 1/3 🔄
▶️ Iteration 1/15 (Round 1): model 1 e lr 0.0015668949432321277
🔀 Random Seed Atual: 100
Epoch: 1 	Training Loss: 1.433616 	Validation Loss: 0.326847
Epoch: 2 	Training Loss: 1.197285 	Validation Loss: 0.263113
Epoch: 3 	Training Loss: 0.983905 	Validation Loss: 0.431060
Epoch: 4 	Training Loss: 0.962569 	Validation Loss: 0.336314
Epoch: 5 	Training Loss: 0.900942 	Validation Loss: 0.252637
Epoch: 6 	Training Loss: 0.789127 	Validation Loss: 0.228569
Epoch: 7 	Training Loss: 0.718747 	Validation Loss: 0.149641
Epoch: 8 	Training Loss: 0.696015 	Validation Loss: 0.144554
Epoch: 9 	Training Loss: 0.771429 	Validation Loss: 0.179300
Epoch: 10 	Training Loss: 1.031334 	Validation Loss: 0.270777
Epoch: 11 	Training Loss: 0.859582 	Validation Loss: 0.224003


Exception ignored in: <function _MultiProcessingDataLoaderIter.__del__ at 0x0000019E93FCA660>
Traceback (most recent call last):
  File "C:\Users\Lyanh\AppData\Roaming\Python\Python312\site-packages\torch\utils\data\dataloader.py", line 1479, in __del__
    self._shutdown_workers()
  File "C:\Users\Lyanh\AppData\Roaming\Python\Python312\site-packages\torch\utils\data\dataloader.py", line 1437, in _shutdown_workers
    if self._persistent_workers or self._workers_status[worker_id]:
                                   ^^^^^^^^^^^^^^^^^^^^
AttributeError: '_MultiProcessingDataLoaderIter' object has no attribute '_workers_status'


Epoch: 12 	Training Loss: 0.699431 	Validation Loss: 0.139094
Epoch: 13 	Training Loss: 0.647175 	Validation Loss: 0.145645
Epoch: 14 	Training Loss: 0.591448 	Validation Loss: 0.299325
Epoch: 15 	Training Loss: 1.250400 	Validation Loss: 0.251481
Epoch: 16 	Training Loss: 0.859407 	Validation Loss: 0.199170
Epoch: 17 	Training Loss: 0.695167 	Validation Loss: 0.159630
Early stopping after 17 epochs due to no improvement.

🔎 INICIANDO A AVALIAÇÃO DO OACE PARA ITERAÇÃO ATUAL
iteration_metrics:  {'model_name': 'VGG16', 'assertividade': {'precision': 0.7618229307836331, 'accuracy': 0.7569, 'recall': 0.7569000000000001}, 'custo': {'mtp': 134309962, 'tpi': 0.06843847226185404, 'ms': 512.3518447875977}, 'solution': {'lr': 0.0015668949432321277, 'model_index': 1}}
accumulated_metrics:  [{'model_name': 'VGG16', 'assertividade': {'precision': 0.7618229307836331, 'accuracy': 0.7569, 'recall': 0.7569000000000001}, 'custo': {'mtp': 134309962, 'tpi': 0.06843847226185404, 'ms': 512.3518447875977}, '

KeyboardInterrupt: 

Resultados

In [43]:
print(f"Best Model: {best_model}")
print(f"Best Score: {best_score}")
print(f"Best Solution: {best_solution}")
print(f"ML Metrics: {metrics}")
print(f"OACE Metrics: {oace_metrics}")
print(f"Método Finalizado: ")

Best Model: MobileNetV2
Best Score: 0.9355926436048008
Best Solution: [0.004915491436617374, 1]
ML Metrics: {0: {'model_name': 'EfficientNetB0', 'assertividade': {'precision': 0.3931453671403046, 'accuracy': 0.40476190476190477, 'recall': 0.36750230069305684}, 'custo': {'mtp': 4015234, 'tpi': 0.03408905863761902, 'ms': 15.316902160644531}, 'solution': {'lr': 0.008173306304572338, 'model_index': 0}}, 1: {'model_name': 'ResNet50', 'assertividade': {'precision': 0.3096171802054155, 'accuracy': 0.23015873015873015, 'recall': 0.1788377965703547}, 'custo': {'mtp': 23520326, 'tpi': 0.09884101152420044, 'ms': 89.7229232788086}, 'solution': {'lr': 0.008169279428953099, 'model_index': 2}}, 2: {'model_name': 'MobileNetV2', 'assertividade': {'precision': 0.23508678539166347, 'accuracy': 0.3373015873015873, 'recall': 0.29257386810470737}, 'custo': {'mtp': 2231558, 'tpi': 0.04945465922355652, 'ms': 8.512718200683594}, 'solution': {'lr': 0.007750224415955002, 'model_index': 1}}, 3: {'model_name': 'Ef

In [44]:
metrics

{0: {'model_name': 'EfficientNetB0',
  'assertividade': {'precision': 0.3931453671403046,
   'accuracy': 0.40476190476190477,
   'recall': 0.36750230069305684},
  'custo': {'mtp': 4015234,
   'tpi': 0.03408905863761902,
   'ms': 15.316902160644531},
  'solution': {'lr': 0.008173306304572338, 'model_index': 0}},
 1: {'model_name': 'ResNet50',
  'assertividade': {'precision': 0.3096171802054155,
   'accuracy': 0.23015873015873015,
   'recall': 0.1788377965703547},
  'custo': {'mtp': 23520326,
   'tpi': 0.09884101152420044,
   'ms': 89.7229232788086},
  'solution': {'lr': 0.008169279428953099, 'model_index': 2}},
 2: {'model_name': 'MobileNetV2',
  'assertividade': {'precision': 0.23508678539166347,
   'accuracy': 0.3373015873015873,
   'recall': 0.29257386810470737},
  'custo': {'mtp': 2231558,
   'tpi': 0.04945465922355652,
   'ms': 8.512718200683594},
  'solution': {'lr': 0.007750224415955002, 'model_index': 1}},
 3: {'model_name': 'EfficientNetB0',
  'assertividade': {'precision': 0.3

In [46]:
oace_metrics

{0: {'model_name': 'EfficientNetB0',
  'A': 0.0,
  'C': 0.746754924222813,
  'Score': 0.5600661931671097,
  'solution': {'lr': 0.008173306304572338, 'model_index': 0}},
 1: {'model_name': 'ResNet50',
  'A': 0.0,
  'C': 0.031222735351751237,
  'Score': 0.023417051513813427,
  'solution': {'lr': 0.008169279428953099, 'model_index': 2}},
 2: {'model_name': 'MobileNetV2',
  'A': 0.16419434692639776,
  'C': 0.8119999999999999,
  'Score': 0.6500485867315994,
  'solution': {'lr': 0.007750224415955002, 'model_index': 1}},
 3: {'model_name': 'EfficientNetB0',
  'A': 0.8004349448738249,
  'C': 0.746754924222813,
  'Score': 0.760174929385566,
  'solution': {'lr': 0.008491695916522522, 'model_index': 0}},
 4: {'model_name': 'MobileNetV2',
  'A': 0.5184705903013165,
  'C': 0.8119999999999999,
  'Score': 0.7386176475753291,
  'solution': {'lr': 0.008690641876341252, 'model_index': 1}},
 5: {'model_name': 'ResNet50',
  'A': 1.0231979099304807,
  'C': 0.031222735351751237,
  'Score': 0.279216528996433

Salvando métricas

In [ ]:
# Salvando as métricas
with open('ML_metrics.json', 'w') as f:
    json.dump(metrics, f)
with open('OACE_metrics.json', 'w') as f:
    json.dump(oace_metrics, f)